In [ ]:
# default_exp shortest_path

# Shortest path

> subtitle tbd

edge weight = cost of a path


weights unequal --> use Djistra's algo

![](img/1.png)
![](img/2.png)

In [ ]:
# export
from queue import Queue
from typing import List, Dict
from collections import namedtuple

import pandas as pd

import graph_utils.core as gu

In [ ]:
# export
Row = namedtuple('Node', ['distance_from_source', 'preceding_vertex'])

In [ ]:
?Row

Init signature: Row(distance_from_source, preceding_vertex)
Docstring:      Node(distance_from_source, preceding_vertex)
Type:           type
Subclasses:     


In [ ]:
# export
def _distance_table_to_df(distance_table:Dict):
    df = pd.DataFrame.from_dict(distance_table, orient='index')
    df.index.name = 'vertex_id'
    return df

In [ ]:
# export
def build_distance_table(graph:gu.Graph, source:int):
    distance_table = dict()

    # initiate an empty distance table
    for i in range(graph.numVertices):
        #distance_table[i] = Row(distance_from_source=None, preceding_vertex=None)
        distance_table[i] = Row(None, None)

    # distance to the source from itself is 0
    distance_table[source] = Row(0, source)

    queue = Queue()
    queue.put(source)

    while not queue.empty():
        current_vertex = queue.get()

        # the distance of current vertex from the source
        current_distance = distance_table[current_vertex][0] # zero is index of distance

        for neighbor in graph.get_adjacent_vertices(current_vertex):
            # only update dist table if no current distance from the source is set
            neigbor_was_visited = distance_table[neighbor][0] is not None
            if not neigbor_was_visited:
                distance_table[neighbor] = Row(current_distance + 1, current_vertex)

                # enqueue the neigbor only if in has other adjacent vertices to explore
                if len(graph.get_adjacent_vertices(neighbor)):
                    queue.put(neighbor)





    return distance_table

In [ ]:
?Row

Init signature: Row(distance_from_source, preceding_vertex)
Docstring:      Node(distance_from_source, preceding_vertex)
Type:           type
Subclasses:     


In [ ]:
# export

def shortest_path(graph:gu.Graph, source:int, destination:int):
    distance_table = build_distance_table(graph, source)
    path = [destination]
    
    # distance_table is a dict, s.t
    #                       0                     1
    # key:int = Row(distance_from_source, preceding_vertex)
    
    previous_vertex = distance_table[destination][1]
    
    while previous_vertex is not None and previous_vertex is not source:
        path = [previous_vertex] + path
        previous_vertex = distance_table[previous_vertex][1]
    
    if previous_vertex is None:
        msg = 'There is no path from {} to {}'.format(source, destination)
        print(msg)
        return None
    else:
        path = [source] + path
        return path

![](img/3.png)

In [ ]:
g = gu.AdjacencySetGraph(8, directed=False)
g.add_edge(0, 1)
g.add_edge(1, 2)
g.add_edge(1, 3)
g.add_edge(2, 3)
g.add_edge(1, 4)
g.add_edge(3, 5)
g.add_edge(5, 4)
g.add_edge(3, 6)
g.add_edge(6, 7)
g.add_edge(0, 7)
g

0 --> 1
0 --> 7
1 --> 0
1 --> 2
1 --> 3
1 --> 4
2 --> 1
2 --> 3
3 --> 1
3 --> 2
3 --> 5
3 --> 6
4 --> 1
4 --> 5
5 --> 3
5 --> 4
6 --> 3
6 --> 7
7 --> 0
7 --> 6

In [ ]:
build_distance_table(g, 0)


_distance_table_to_df(build_distance_table(g, 0))

,distance_from_source,preceding_vertex
vertex_id,,
0,0,0
1,1,0
2,2,1
3,2,1
4,2,1
5,3,3
6,2,7
7,1,0


In [ ]:
_distance_table_to_df(build_distance_table(g, 0))
expected_dict = {'distance_from_source': {0: 0, 1: 1, 2: 2, 3: 2, 4: 2, 5: 3, 6: 2, 7: 1},
                     'preceding_vertex': {0: 0, 1: 0, 2: 1, 3: 1, 4: 1, 5: 3, 6: 7, 7: 0}
                }

expected = pd.DataFrame(expected_dict)
got = _distance_table_to_df(build_distance_table(g, 0))
comparison_matrix = (got == expected)
not_equal = ~comparison_matrix.all(axis=1)

if not_equal.any():
    raise ValueError(f"{not_equal.sum()} vertices in distance table do not match")

In [ ]:
assert shortest_path(g, 0, 5) == [0,1,3,5]
assert shortest_path(g, 0, 6) == [0,7,6]
assert shortest_path(g, 7, 4) == [7,0,1,4]

In [ ]:
g = gu.AdjacencySetGraph(8, directed=True)
g.add_edge(0, 1)
g.add_edge(1, 2)
g.add_edge(1, 3)
g.add_edge(2, 3)
g.add_edge(1, 4)
g.add_edge(3, 5)
g.add_edge(5, 4)
g.add_edge(3, 6)
g.add_edge(6, 7)
g.add_edge(0, 7)

assert shortest_path(g, 0, 5) == [0,1,3,5]
assert shortest_path(g, 0, 6) == [0,1,3,6]
assert shortest_path(g, 7, 4) is None

There is no path from 7 to 4
